In [1]:
import pandas as pd
pd.options.mode.copy_on_write = True
from datetime import datetime, timedelta
import secrets
import string
import re  # Import the regular expression module
import pandas as pd
import requests
import zipfile
from io import BytesIO

In [1]:
import pandas as pd

In [7]:
df = pd.read_csv("/home/mohammadp/RenewableInsight/data/produkt_tu_stunde_20221013_20240414_00044.txt",delimiter=";")

In [8]:
df.head(5)

,STATIONS_ID,MESS_DATUM,QN_9,TT_TU,RF_TU,eor
0,44,2022101300,3,6.9,94.0,eor
1,44,2022101301,3,7.3,93.0,eor
2,44,2022101302,3,6.6,93.0,eor
3,44,2022101303,3,7.8,91.0,eor
4,44,2022101304,3,8.3,91.0,eor


In [9]:
df.drop(columns=['QN_9','eor','RF_TU'],inplace=True)

In [28]:
df.head(5)

,STATIONS_ID,MESS_DATUM,TT_TU,air_temperature
0,44,2022-10-13 00:00:00,6,6
1,44,2022-10-13 01:00:00,7,7
2,44,2022-10-13 02:00:00,6,6
3,44,2022-10-13 03:00:00,7,7
4,44,2022-10-13 04:00:00,8,8


In [21]:
max(df.TT_TU)

32.4

In [27]:
df.dtypes

STATIONS_ID                 int16
MESS_DATUM         datetime64[ns]
TT_TU                       int16
air_temperature             int16
dtype: object

In [12]:
df.shape

(13200, 3)

In [16]:
df.MESS_DATUM = df.MESS_DATUM.astype('str')

In [18]:
df.MESS_DATUM = pd.to_datetime(df.MESS_DATUM, format='%Y%m%d%H')

In [22]:
df.TT_TU = df.TT_TU.astype('int16')

In [24]:
df.STATIONS_ID = df.STATIONS_ID.astype('int16')

In [26]:
df['air_temperature'] = df.TT_TU

In [29]:
df.drop(columns = ['TT_TU'])

,STATIONS_ID,MESS_DATUM,air_temperature
0,44,2022-10-13 00:00:00,6
1,44,2022-10-13 01:00:00,7
2,44,2022-10-13 02:00:00,6
3,44,2022-10-13 03:00:00,7
4,44,2022-10-13 04:00:00,8
...,...,...,...
13195,44,2024-04-14 19:00:00,10
13196,44,2024-04-14 20:00:00,8
13197,44,2024-04-14 21:00:00,8
13198,44,2024-04-14 22:00:00,6


In [2]:
from enum import Enum
import requests
import zipfile
from io import BytesIO
import pandas as pd

class WeatherParameter(Enum):
    TU = ("air_temperature", "Temperature in °C", ['QN_9', 'eor', 'RF_TU'], '_akt')
    SD = ("sun", "Sunshine Duration", ['QN_9', 'eor', 'RF_SD'], '_akt')
    FF = ("wind", "Wind Speed in km/h", ['QN_9', 'eor', 'RF_FF'], '_akt')
    ST = ("solar", "Solar Radiation", ['QN_9', 'eor', 'RF_ST'], '_row')  # Special case for Solar Radiation
    F = ("wind_synop", "Synoptic Wind Data", ['QN_9', 'eor', 'RF_F'], '_akt')
    
    def __init__(self, category, description, columns, url_suffix):
        self.category = category
        self.description = description
        self.columns = columns
        self.url_suffix = url_suffix

class WeatherDataDownloader:
    def __init__(self, base_url=None):
        self.base_url = base_url or "https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/hourly/"

    def download_and_load_weather_data(self, station_code, weather_param):
        if not isinstance(weather_param, WeatherParameter):
            raise ValueError("weather_param must be an instance of WeatherParameter Enum.")
        
        category = weather_param.category
        suffix = weather_param.url_suffix
        if weather_param == WeatherParameter.ST:
            url = f"{self.base_url}{category}/stundenwerte_{weather_param.name}_{station_code}{suffix}.zip" # Special case for Solar Radiation
        else:
            url = f"{self.base_url}{category}/recent/stundenwerte_{weather_param.name}_{station_code}{suffix}.zip" 
        response = requests.get(url)
        response.raise_for_status()
        with zipfile.ZipFile(BytesIO(response.content)) as thezip:
            # Dynamically find the correct file based on the pattern
            pattern = re.compile(f"produkt_{weather_param.name.lower()}_stunde_.*_{station_code}.txt")
            file_name = next((s for s in thezip.namelist() if pattern.match(s)), None)
            if not file_name:
                raise FileNotFoundError(f"No file matching '{pattern.pattern}' found in the ZIP archive.")
            with thezip.open(file_name) as file:
                df = pd.read_csv(file, delimiter=';', encoding='latin1')
        return df

In [4]:
WeatherParameter['TU'].category

'air_temperature'

In [55]:
# Example Usage:
downloader = WeatherDataDownloader()
station_code = '00691'
weather_param = WeatherParameter.ST # Using the Enum here
df = downloader.download_and_load_weather_data(station_code, weather_param)
if df is not None:
    print(df.head())
else:
    print("Data loading failed.")

   STATIONS_ID     MESS_DATUM  QN_592  ATMO_LBERG  FD_LBERG  FG_LBERG  \
0          691  2009010100:28       1        -999       0.0       0.0   
1          691  2009010101:28       1        -999       0.0       0.0   
2          691  2009010102:28       1        -999       0.0       0.0   
3          691  2009010103:28       1        -999       0.0       0.0   
4          691  2009010104:28       1        -999       0.0       0.0   

   SD_LBERG   ZENIT MESS_DATUM_WOZ  eor  
0         0  149.43  2009010101:00  eor  
1         0  145.44  2009010102:00  eor  
2         0  138.70  2009010103:00  eor  
3         0  130.48  2009010104:00  eor  
4         0  121.60  2009010105:00  eor  


In [56]:
df.head(5)

,STATIONS_ID,MESS_DATUM,QN_592,ATMO_LBERG,FD_LBERG,FG_LBERG,SD_LBERG,ZENIT,MESS_DATUM_WOZ,eor
0,691,2009010100:28,1,-999,0.0,0.0,0,149.43,2009010101:00,eor
1,691,2009010101:28,1,-999,0.0,0.0,0,145.44,2009010102:00,eor
2,691,2009010102:28,1,-999,0.0,0.0,0,138.70,2009010103:00,eor
3,691,2009010103:28,1,-999,0.0,0.0,0,130.48,2009010104:00,eor
4,691,2009010104:28,1,-999,0.0,0.0,0,121.60,2009010105:00,eor


In [22]:
# Example Usage:
downloader = WeatherDataDownloader()
station_code = '00691'
weather_param = WeatherParameter.ST # Using the Enum here
df = downloader.download_and_load_weather_data(station_code, weather_param)
if df is not None:
    print(df.head())
else:
    print("Data loading failed.")

   STATIONS_ID     MESS_DATUM  QN_592  ATMO_LBERG  FD_LBERG  FG_LBERG  \
0          691  2009010100:28       1        -999       0.0       0.0   
1          691  2009010101:28       1        -999       0.0       0.0   
2          691  2009010102:28       1        -999       0.0       0.0   
3          691  2009010103:28       1        -999       0.0       0.0   
4          691  2009010104:28       1        -999       0.0       0.0   

   SD_LBERG   ZENIT MESS_DATUM_WOZ  eor  
0         0  149.43  2009010101:00  eor  
1         0  145.44  2009010102:00  eor  
2         0  138.70  2009010103:00  eor  
3         0  130.48  2009010104:00  eor  
4         0  121.60  2009010105:00  eor  


In [57]:
df.dtypes

STATIONS_ID         int64
MESS_DATUM         object
QN_592              int64
ATMO_LBERG          int64
FD_LBERG          float64
FG_LBERG          float64
SD_LBERG            int64
ZENIT             float64
MESS_DATUM_WOZ     object
eor                object
dtype: object

In [5]:
df.ATMO_LBERG.unique()

array([-999])

In [24]:
# Step 1: Convert the 'MESS_DATUM' column to datetime, ignoring the minutes
df['MESS_DATUM'] = pd.to_datetime(df['MESS_DATUM'].str.slice(stop=10), format='%Y%m%d%H')


In [25]:
df

,STATIONS_ID,MESS_DATUM,QN_592,ATMO_LBERG,FD_LBERG,FG_LBERG,SD_LBERG,ZENIT,MESS_DATUM_WOZ,eor
0,691,2009-01-01 00:00:00,1,-999,0.0,0.0,0,149.43,2009010101:00,eor
1,691,2009-01-01 01:00:00,1,-999,0.0,0.0,0,145.44,2009010102:00,eor
2,691,2009-01-01 02:00:00,1,-999,0.0,0.0,0,138.70,2009010103:00,eor
3,691,2009-01-01 03:00:00,1,-999,0.0,0.0,0,130.48,2009010104:00,eor
4,691,2009-01-01 04:00:00,1,-999,0.0,0.0,0,121.60,2009010105:00,eor
...,...,...,...,...,...,...,...,...,...,...
133759,691,2024-03-31 19:00:00,1,-999,0.0,0.0,0,99.53,2024033120:00,eor
133760,691,2024-03-31 20:00:00,1,-999,0.0,0.0,0,107.51,2024033121:00,eor
133761,691,2024-03-31 21:00:00,1,-999,0.0,0.0,0,114.29,2024033122:00,eor
133762,691,2024-03-31 22:00:00,1,-999,0.0,0.0,0,119.29,2024033123:00,eor
